# Exploring Hacker News Posts

[Hacker News](https://news.ycombinator.com/) is a popular site where users submit technology related posts that can be voted and commented upon.

We are going to work with a dataset of submissions to Hacker News, specifically those with titles that begin with either `Ask HN` or `Show HN`. Users submit `Ask HN` posts to ask the community a question, such as "How can I further improve the accessibility of my website?". Users submit `Show HN` posts to show the community a project, product or just generally something interesting.

We'll compare these two types of posts to determine the following:

- Do `Ask HN` or `Show HN` receive more comments on average?
- Do posts created at a certain time receive more comments on average?

The dataset can be found [here](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts), however do note that it has been reduced from approx 300,000 rows to approx 20,000 rows for this project. The rows were reduced by removing all submissions that did not receive any comments, and then random sampling from the remaining submissions.